# Optimization Algorithms Challenge

---

1) QUBO formulation for the Knapsack Problem (KP)

    1.1) Give the analytical description of the QUBO formulation for the Knapsack Problem

    $$Q = -\sum_{i=0}^N v_i q_i + \alpha\left(\sum_{i,j=0}^N w_i w_j q_i q_j - 2\sum_{i=0}^N\sum_{j=0}^{\lfloor \log{2}{W} \rfloor}w_i a_j q_i s_j - +\sum_{i,j=0}^{\lfloor \log{2}{W} \rfloor} a_i a_j s_i s_j\right)$$

    where $\alpha = max\;  v_i +1$ and $a_j = \begin{cases} 2^j & j<\lfloor \log{2}{W} \rfloor \\ W-2^{\lfloor \log{2}{W} \rfloor}+1 & j=\lfloor \log{2}{W} \rfloor \end{cases}$. 

    1.2) Find the optimal number of slack variables and their coefficients $a_i$ for any range (assume that $S$ has a range $[0, M]$)

    Given that $S = \sum_{i=0}^{n_{slack}}a_i s_i \in [0,W]$, we can do a decimal-to-binary encoding (sometimes referred to as the "log trick"), and encode the weight in binary variables. Thus, we use $\log{2}{W}$ slack variables and have $a_i = 2^i$. Nonetheless, this would give us only the option to represent a maximum weight that is a power of 2 minus 1, so we need to modify the last parameter, $a_{\log{2}{W}}$, and force it to match the maximum weight as $a_i = W-2^{\lfloor \log{2}{W} \rfloor}+1$.

    1.3) Find the minimum value for the Lagrange multiplier in order to ensure the problem constraints.

    Consider we go over the maximum weight by one object with $w_i = 1$. This object will contribute with at most $-v_{max}$ to Q, so we have to penalize it with some $\alpha>v_{max}$, such as $\alpha = max \; v_i + 1$.

    1.4) Write a code that given a KP instance, automatically generates the QUBO matrix as the weights matrix $Q$ from the equation above.

    1.5) Write a function that given a solution, returns the value for the slack variables that fulfill the constraint (if possible).

    1.6) Code a function that given a bitstring solution, returns the associated cost function value using the Hamiltonian matrix.

    Optional:

    1.7) Extend the formulation to include the following dependencies between the items accepted by the solution:

    - If items $i$ and $j$ are included, the total value is increased by an additional quantity.

    $-\Delta q_i q_j$, where $\Delta$ is an additional quantity.
        
    - If one of the items $i$ and $j$ is included without the other, the total value is decreased by a given quantity.

    $(1-q_i q_j)(1-(1-q_i)(1-q_j))$, which is an implementation of the XOR gate, and can be further simplified to quadratic form taking into account that $q_i^2 = q_i$.


In [ ]:
# 1.4
import numpy as np
def get_Knapsack_QUBO(v, w, W):
    # v is the vector of values of each item
    # w is the vector of weights of each item
    # W is the maximum weight the knapsack can hold
    N = len(v)
    Q = np.zeros((N+int(np.log2(W))+1,N+int(np.log2(W))+1))
    alpha = max(v) + 1
    # Calculate the QUBO matrix
    for i in range(N):
        for j in range(N):
            if i == j:
                Q[(i,j)] = -v[i] + alpha*w[i]**2
            else:
                Q[(i,j)] = alpha*w[i]*w[j]
    for i in range(N):
        for j in range(int(np.log2(W))+1):
            if j < int(np.log2(W)):
                Q[(i,N+j)] = -2*alpha*w[i]*2**j
                Q[(N+j,i)] = -2*alpha*w[i]*2**j
            else:
                Q[(i,N+j)] = -2*alpha*w[i]*(W-2**int(np.log2(W))+1)
                Q[(N+j,i)] = -2*alpha*w[i]*(W-2**int(np.log2(W))+1)
    for i in range(int(np.log2(W))+1):
        for j in range(int(np.log2(W))+1):
            if i < int(np.log2(W)) and j < int(np.log2(W)):
                Q[(N+i,N+j)] = alpha*2**(i+j)
                Q[(N+j,N+i)] = alpha*2**(i+j)
            elif i < int(np.log2(W)) and j == int(np.log2(W)):
                Q[(N+i,N+j)] = alpha*(W-2**int(np.log2(W))+1)*2**i
                Q[(N+j,N+i)] = alpha*(W-2**int(np.log2(W))+1)*2**i
            elif i == int(np.log2(W)) and j < int(np.log2(W)):
                Q[(N+i,N+j)] = alpha*(W-2**int(np.log2(W))+1)*2**j
                Q[(N+j,N+i)] = alpha*(W-2**int(np.log2(W))+1)*2**j
            else:
                Q[(N+i,N+j)] = alpha*(W-2**int(np.log2(W))+1)**2
                Q[(N+j,N+i)] = alpha*(W-2**int(np.log2(W))+1)**2
    return Q

In [ ]:
#1.5
import math as m


def slack_values(W, maxW):
    n_bits = int(m.log(maxW,2))
    val_coef_extra = maxW - pow(2,int(m.log(maxW,2))) + 1

    val_bit_extra = 0
    if W >= val_coef_extra:
        val_bit_extra = 1
    
    W = W - val_bit_extra*val_coef_extra

    W += val_bit_extra * 2**n_bits

    return bin(W)



print(slack_values(1 , 2))

In [ ]:
# 1.6
def cost(solution,v):
    # solution is a bitstring representing the solution, where 1 means the item is taken and 0 means it is not
    # v is the vector of values of each item, in the same order as the solution
    return -sum([solution[i]*v[i] for i in range(len(solution))])

# Variational Quantum Algorithm
---

2) Variational Quantum Eigensolver for the KP

    2.1) Define and code a hardware-efficient ansatz suitable for a line connectivity ($q_0-q_1-...-q_{n-1}$) with an arbitrary number of layers $p$.

    2.2) Write a function that given a set of parameters $\vec{\theta}$, an anstaz and a cost function, runs the circuit and evaluates the cost function by measuring $n\_shots$ times.

    2.3) Code the workflow of the entire variational algorithm where the circuit is executed and the parameters are updated iteratively using a Scipy optimizer (e.g. Powell, BFGS, Newton-CG, etc.)

    2.4) Run simulations with different number of ansatz layers to check the scalability of the algorithm.

    The user can change the value of p in our code, and see that as p grows larger, in general, the results will improve, up to a point of saturation. This is due to the fact that adding layers to the ansatz increases its expressibility (ability to explore the Hilbert space), but this may lead to barren plateaus (exponential decay of the variance of differences of cost function values, that is, the cost landscape flattens exponentially).

    Optional:

    2.5) Try other type of optimizers (e.g. CMA-ES evolutionary optimizer)

    We tried BFGS but didn't get good results.

    2.6) Look for a strategy to select the initial values of the variational parameters that improves the performance of the algorithm.

    A possible strategy is to have a guess of some solutions (or close-to) and take a superposition of them as the initial state. Another possibility is to take only one guess of solution, but instead of starting right from that one (for some gradient-based algorithms this will lead to the algorithm getting stuck in this local minimum), make a small perturbation (or warm start).

---


In [ ]:
from qibo import gates, models
import numpy as np
import scipy as sp

def HEA(nqubits,p,parameters):
    qc = models.Circuit(nqubits)
    for k in range(p):
        for i in range(nqubits):
            qc.add(gates.RY(i,parameters[i,k]))
        for i in range(nqubits-1):
            qc.add(gates.CNOT(i,i+1))
    return qc

def cost(solution):
    Q = get_Knapsack_QUBO(v, w, W)
    return np.dot(solution,np.dot(Q,solution))
        
def run_ansatz(ansatz,parameters,costfunction,n_shots,return_samples=False):
    nqubits = len(parameters)
    p = parameters[0].size
    parametrized_circuit = ansatz(nqubits,p,parameters)
    parametrized_circuit.add(gates.M(i) for i in range(nqubits))

    result = parametrized_circuit(nshots=n_shots)
    samples = result.samples()
    cost = 0
    for sample in samples:
        cost += costfunction(sample)/n_shots
    if return_samples:
        return cost,list(np.max(result.frequencies()).keys())[0]
    else:
        return cost

def ansatz_to_optimize(parameters):
    parameters = parameters.reshape((n+s,p))
    return run_ansatz(HEA,parameters,cost,100,False)

# test for some instance
v = [4,5,2,4,2,5]
w = [4,3,5,1,3,3]
W = 7
s = int(np.log2(W))+1
n = len(v)
p = 5
initial_parameters = np.random.rand((n+s)*p)*2*np.pi
optimal_params = sp.optimize.minimize(ansatz_to_optimize,initial_parameters,method='Powell')
parameters = optimal_params.x.reshape((n+s,p))
optimal_cost, best_sample = run_ansatz(HEA,parameters,cost,100,True)
print(best_sample, optimal_cost)


3) Quantum Annealing (QA)

    3.1) Construct the `qibo` problem Hamiltonian object from the QUBO matrix and the mixer Hamiltonian $H_M = \sum_i^n \sigma^x_i$

    3.2) Write a function that performs an annealing evolution with a linear schedule and evaluates the cost function with the final state

    3.3) Study how the total annealing time $T$ and the time step $dt$ affect the quantum state evolution and the quality of the final result

    3.4) Study how the value of the Lagrange multipliers affects the final result

    Optional:

    3.5) Define a quadratic scheduling function with free parameters and find the optimal coeffcients with a classical optimizer.

---

4) From QA to VQAs

    4.1) Find online and implement the Variational Quantum Algorithm that is inspired by digitalized version of QA.

    4.2) Study the scaling and parameters selection as done in previous sections.

    4.3) Compare (qualitatively or quantitatively) this algorithm with the VQE and QA algorithms.

---

5) Towards more complex algorithms

    Explore and implement variations of the previous algorithms and compare them with the original ones. Choose the algorithm of your will, here you can find some suggestions:

    Analog:

    - Reverse quantum annealing

    - Quantum annealing with counterdiabatic driving

    Digital:

    - Layer VQE (L-VQE)

    - Linear Ramp QAOA (LR-QAOA)

    - ADAPT-VQE